### Script de preprocesamiento de dataset de entrenamiento

Script ilustrativo (no comentado), que sirve para mostrar el preprocesamiento usado para obtener el conjunto de entrenamiento

In [1]:
import requests
import numpy as np
import json
from bs4 import BeautifulSoup
import re
import os
import urllib.request
import shutil
import pandas as pd
import csv
from itertools import islice
import matplotlib.pyplot as plt

In [2]:
stations=['CCA', 'CCHA', 'CCHN', 'CCHO', 'CCHS', 'CCHV', 'ENP1', 'ENP2', 'ENP3', 'ENP4', 'ENP5', 'ENP6', 'ENP7', 'ENP8', 'ENP9']
# Latitudes
lats = [19.3268, 19.4964, 19.4663, 19.3581, 19.3050, 19.4844, 19.4381,\
             19.3470, 19.4658, 19.4044, 19.4154, 19.4206, 19.3281, 19.4821, 19.3618]
# Longitudes
lons = [-99.1761, -99.1761, -99.2462, -99.0199, -99.1815, -99.1413,\
              -99.1330, -99.0986, -99.1273, -99.1572, -99.1871, -99.1586, -99.0745, -99.1338, -99.0602]
# Alturas
alts = [2277, 2240, 2280, 2230, 2290, 2242, 2248, 2250, 2270, 2265, 2245, 2275, 2235, 2255, 2225]

In [3]:
pwd=os.getcwd()
path_db=pwd+"/../data"
suma=0
df_list=[]
for ind,d in enumerate(stations):
    path_station="%s/%s"%(path_db,d)
    print(d)
    suma+=1
    for f in sorted(os.listdir(path_station)): 
        if f != ".DS_Store" and (int(f[5:7])>4) and (int(f[5:7])<11):
            path_f="%s/%s"%(path_station,f)
            print(f)
            df=pd.read_csv(path_f,skiprows=7)
            df=df.drop(index=0)
            try:
                df = df.drop(columns=["Dosis_UV"])
            except:
                pass
                #print("no existe columna")
            try:
                df = df.drop(columns=["Dosis_UV                                  "])
            except:
                pass
            df['Estacion']=d
            df['Latitud']=lats[ind]
            df['Longitud']=lons[ind]
            df['Altitud']=alts[ind]
            df = df[df.columns[-4:].tolist()+ df.columns[:-4].tolist()]
            #se quitan espacios en blanco
            df=df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
            df_list.append(df)
            print(df.columns.values)
            

CCA
2007-05-CCA-L1.csv
['Estacion' 'Latitud' 'Longitud' 'Altitud' 'Fecha_hora' 'Temp' 'Hum_Rel'
 'Rapidez_v_sostenido' 'Dir_v_sostenido' 'Rapidez_rachas' 'Dir_rachas'
 'Presion_bar' 'Precipitacion' 'Rad_Solar' 'Indice_UV']
2007-06-CCA-L1.csv
['Estacion' 'Latitud' 'Longitud' 'Altitud' 'Fecha_hora' 'Temp' 'Hum_Rel'
 'Rapidez_v_sostenido' 'Dir_v_sostenido' 'Rapidez_rachas' 'Dir_rachas'
 'Presion_bar' 'Precipitacion' 'Rad_Solar' 'Indice_UV']
2007-07-CCA-L1.csv
['Estacion' 'Latitud' 'Longitud' 'Altitud' 'Fecha_hora' 'Temp' 'Hum_Rel'
 'Rapidez_v_sostenido' 'Dir_v_sostenido' 'Rapidez_rachas' 'Dir_rachas'
 'Presion_bar' 'Precipitacion' 'Rad_Solar' 'Indice_UV']
2007-08-CCA-L1.csv
['Estacion' 'Latitud' 'Longitud' 'Altitud' 'Fecha_hora' 'Temp' 'Hum_Rel'
 'Rapidez_v_sostenido' 'Dir_v_sostenido' 'Rapidez_rachas' 'Dir_rachas'
 'Presion_bar' 'Precipitacion' 'Rad_Solar' 'Indice_UV']
2007-09-CCA-L1.csv
['Estacion' 'Latitud' 'Longitud' 'Altitud' 'Fecha_hora' 'Temp' 'Hum_Rel'
 'Rapidez_v_sostenido' 'Dir_

In [4]:
df_all= pd.concat(df_list, ignore_index=True)

In [5]:
def convertir_fecha(fecha_str):
    try:
        return pd.to_datetime(fecha_str, format='%Y/%m/%d %H:%M:%S')
    except ValueError:
        return pd.to_datetime(fecha_str, format='%d/%m/%y %H:%M', errors='coerce')
df_all['Fecha_hora'] = df_all['Fecha_hora'].apply(convertir_fecha)

In [6]:
df_all["Precipitacion"] = pd.to_numeric(df_all["Precipitacion"],errors='coerce')

In [7]:
df_all.shape

(2615313, 15)

In [8]:
def rain_next_hour(df_group):
    # se ordena el df por la columna de fechas
    df_group = df_group.sort_values('Fecha_hora').reset_index(drop=True)
    
    # se crea un df desplazado para simular la siguiente hora
    df_next = df_group.copy()
    df_next['Fecha_hora'] -= pd.Timedelta(hours=1)  # Restar una hora en df_next

    # se realiza merge del df original con el desplazado
    merged = pd.merge(df_group, df_next[['Fecha_hora', 'Precipitacion']], 
                      on='Fecha_hora', suffixes=('', '_next'), how='left')
    
    # se crea la columna 'Precip_sig_hora' para indicar si llovió en la siguiente hora, esta es la variable objetivo
    merged['Precip_sig_hora'] = merged['Precipitacion_next'].apply(lambda x: 1 if x > 0 else (0 if pd.notna(x) else pd.NA))
    
    # se regres el df con la nueva columna, eliminando columna no necesaria
    return merged.drop(columns=['Precipitacion_next'])

# se aplica la función a cada estacion y año
df_all2 = df_all.groupby(['Estacion', pd.PeriodIndex(df_all['Fecha_hora'], freq="Y")]).apply(rain_next_hour).reset_index(drop=True)

df_all2.shape

/var/folders/fw/lfwqgrwx72v8gqhx3vm2nv780000gn/T/ipykernel_2449/2774036250.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_all2 = df_all.groupby(['Estacion', pd.PeriodIndex(df_all['Fecha_hora'], freq="Y")]).apply(rain_next_hour).reset_index(drop=True)


(2653509, 16)

In [9]:
path_final_csv="%s/%s"%(path_db,'rain_pembu3.csv')
df_all2.to_csv(path_final_csv,index=False)